In [3]:
from matplotlib import pyplot as plt

from path import *

In [4]:
observations = pd.read_csv(os.path.join(databases_dir, "correlations.csv"))

observations = observations.dropna(how="any", axis="index", subset=corr_columns)
observations = observations.drop(columns=["Unnamed: 0", 'Heading', 'Lat', 'Lon', 
                                          "corr_on_threshold_0.6", "corr_on_threshold_0.7", 'corr_on_threshold_0.9', 'corr_on_threshold_0.95', 'corr_on_threshold_0.99',
                                         'photo_on_threshold_0.6', 'photo_on_threshold_0.7', 'photo_on_threshold_0.9', 'photo_on_threshold_0.95', 'photo_on_threshold_0.99',])
observations = observations.reset_index(drop=True)

corr_columns = list(corr for corr in observations.columns if corr.startswith("corr_on_threshold"))
photo_columns = list(corr for corr in observations.columns if corr.startswith("photo_on_threshold"))
photo_columns.append("time")

for photo_column in photo_columns:
    observations[photo_column] = pd.to_datetime(observations[photo_column])
    
    
observations.dropna(axis='index', subset=["Types of clouds"], how='any')

NameError: name 'corr_columns' is not defined

In [ ]:
def to_timedelta(row: pd.Series):
    for photo_column in photo_columns:
        row[photo_column] = (row[photo_column] - row["time"]).value
    return row

observations = observations.apply(to_timedelta, axis=1)

observations

In [ ]:
def split_clouds_type(row: pd.Series):
    # extract first base type
    cloud_type = row["Types of clouds"]
    if type(cloud_type) == str:
        main_type = cloud_type[:2]
        clouds = cloud_type.split(",")
        clouds = set(cloud[:2] for cloud in clouds)

        return main_type
    return "-"

In [ ]:
observations["Types of clouds"] = observations.apply(split_clouds_type, axis=1)
uniq_types = pd.DataFrame(set(observations["Types of clouds"]), columns=["Types of clouds"])
uniq_types

In [ ]:
def calc_speed(row: pd.Series):
    sub_observations = observations[observations["Types of clouds"] == row["Types of clouds"]]
        
    for photo_column in photo_columns:
        row["mean_" + photo_column] = pd.Timedelta(sub_observations[photo_column].mean())
        row["max_" + photo_column] = pd.Timedelta(sub_observations[photo_column].max())
        row["min_" + photo_column] = pd.Timedelta(sub_observations[photo_column].min())

    return row
    
    
uniq_types = uniq_types.apply(calc_speed, axis=1)
uniq_types

In [ ]:
uniq_types.to_csv(os.path.join(databases_dir, "correlations_on_types.csv"))